In [1]:
import tensorflow as tf
import numpy as np

def create_dataset():
    featureset = []
    test_size = 0.2
    data_size = 100

    for i in range(data_size):
        features = np.ones((11,12,12,12), dtype=np.float32)
        classification = np.array([1.], dtype=np.float32)
        featureset.append([features, classification])
    
    testing_size = int(test_size*data_size)
    features = np.array(featureset)   
    
    train_x = list(features[:,0][:-testing_size])
    train_y = list(features[:,1][:-testing_size])
    test_x = list(features[:,0][-testing_size:])
    test_y = list(features[:,1][-testing_size:])

    #print (train_x[0], train_y[0], test_x[0], test_y[0])
    #print (len(train_x), len(train_y), len(test_x), len(test_y))
    #print (test_y)
    #print (len(train_x[0]))

    return train_x,train_y,test_x,test_y

create_dataset()

([array([[[[1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           ...,
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.]],
  
          [[1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           ...,
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.]],
  
          [[1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           ...,
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.]],
  
          ...,
  
          [[1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           [1., 1., 1., ..., 1., 1., 1.],
           ...,
           [1., 1., 1., 

In [2]:
import tensorflow as tf

train_x,train_y,test_x,test_y = create_dataset()

n_classes = 1
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')

keep_rate = 0.8
keep_prob = tf.placeholder(tf.float32)

def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')
    #Our output tensor produced by max_pooling2d() the 2x2 filter reduces height and width by 50% each.



def convolutional_neural_network(x):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,11,32])),
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #fc: first element determined by result of 2 layers below.
               'W_fc':tf.Variable(tf.random_normal([3*3*3*64,124])),
               'out':tf.Variable(tf.random_normal([124, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([124])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #The methods in the layers module for creating convolutional and pooling layers for two-dimensional 
    #image data expect input tensors to have a shape of [batch_size, image_height, image_width, channels] by default. 
    x = tf.reshape(x, shape=[-1, 12, 12, 12, 11])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = tf.contrib.layers.batch_norm(conv2)
    conv2 = maxpool3d(conv2)

    fc = tf.reshape(conv2,[-1, 3*3*3*64])
    
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out'])+biases['out']

    return output

def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels=y) )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            i=0
            while i < len(train_x):
                start = i
                end = i+batch_size
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])

                _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                              y: batch_y})
                epoch_loss += c
                i+=batch_size
                
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:test_x, y:test_y}))

train_neural_network(x)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

('Epoch', 0, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 1, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 2, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 3, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 4, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 5, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 6, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 7, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 8, 'completed out of', 10, 'loss:', 0.0)
('Epoch', 9, 'completed out of', 10, 'loss:', 0.0)
('Accuracy:', 1.0)


In [ ]:

saver = tf.train.Saver()
saver.save(sess,"./checkpoint_dir/MyModel")